In [1]:
from pathlib import Path

from osgeo import gdal, osr
import gdal2tiles

gdal.UseExceptions()

In [2]:
INPUT_IMAGE = Path('/img/test_4k.png')
TEMP_IMAGE = Path('/tmp/test_4k_georef.tif')
OUTPUT_DIR = Path('../../tiles/')
TILE_SIZE=512

In [3]:
src_ds = gdal.Open(INPUT_IMAGE)
width, height = src_ds.RasterXSize, src_ds.RasterYSize
assert width == height

In [4]:
# new_width = width * 2**1
# new_height = height * 2**1
# src_ds = gdal.Translate('output.tif', src_ds, width=new_width, height=new_height, resampleAlg='bilinear')

In [5]:
driver = gdal.GetDriverByName('GTiff')
dst_ds = driver.Create(TEMP_IMAGE, width, height, src_ds.RasterCount, gdal.GDT_Byte)

In [6]:
for i in range(1, src_ds.RasterCount + 1):
    band = src_ds.GetRasterBand(i)
    dst_band = dst_ds.GetRasterBand(i)
    data = band.ReadAsArray()
    dst_band.WriteArray(data)

dst_ds.SetGeoTransform((0, 1, 0, 0, 0, -1))  # origin top-left, pixel size 1x1

srs = osr.SpatialReference()
srs.ImportFromEPSG(3857)  # Web Mercator
# srs.ImportFromEPSG(4326)  # WGS 84 datum (World Geodetic System 1984)
dst_ds.SetProjection(srs.ExportToWkt())

dst_ds.FlushCache()

0

In [7]:
gdal2tiles.generate_tiles(TEMP_IMAGE, './tiles/', tile_size=TILE_SIZE)

Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100


In [8]:
if TEMP_IMAGE.exists():
    TEMP_IMAGE.unlink()